In [1]:
import pandas as pd
import datetime
import numpy as np
from sklearn import preprocessing
pd.set_option('display.max_columns', None)
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.utils import shuffle


In [2]:
flights_train = pd.read_csv("../data/flights_sample5pct.csv",low_memory=False)


### To predict the cancellation of the flight

In [3]:
def convert_time(df):
    
    df.fl_date = pd.to_datetime(df.fl_date, format='%Y-%m-%d')

    df['month'] = pd.DatetimeIndex(df.fl_date).month

   # df['day'] = pd.DatetimeIndex(df.fl_date).day

    df['day_of_year'] = df.fl_date.dt.dayofyear

    df['day_of_week'] = df.fl_date.dt.dayofweek

    df['crs_dep_time'] = df['crs_dep_time'].apply(lambda x: '{0:0>4}'.format(x))

    df['dep_hour'] = df['crs_dep_time'].str[:2]

    return df

In [4]:
def bins(df):
    hours = {'00':0,'01':0,'02':0,'03':0,'04':0,'05':0,'06':1,'07':1,'08':1,'09':1,'10':1,'11':2,'12':2,'13':2,'14':2,'15':2,'16':2,'17':3,'18':3,'19':3,'20':4,'21':4,'22':4,'23':4,'24': 4}
    df['dep_part_day'] = df['dep_hour'].map(hours)
    season = {1: 1, 2: 1, 3: 2, 4: 2, 5: 2, 6: 3, 7: 3, 8: 3, 9: 4, 10: 4, 11: 4, 12: 1}
    df['season'] = df['month'].map(season)
    
    return df



In [5]:
flights_train.columns

Index(['Unnamed: 0', 'fl_date', 'mkt_unique_carrier', 'branded_code_share',
       'mkt_carrier', 'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'dep_time',
       'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in',
       'crs_arr_time', 'arr_time', 'arr_delay', 'cancelled',
       'cancellation_code', 'diverted', 'dup', 'crs_elapsed_time',
       'actual_elapsed_time', 'air_time', 'flights', 'distance',
       'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay', 'first_dep_time', 'total_add_gtime',
       'longest_add_gtime', 'no_name'],
      dtype='object')

In [28]:
ori=flights_train['origin'].unique()
ori.sort()
ori

array(['ABE', 'ABI', 'ABQ', 'ABR', 'ABY', 'ACK', 'ACT', 'ACV', 'ACY',
       'ADK', 'ADQ', 'AEX', 'AGS', 'AKN', 'ALB', 'ALO', 'ALW', 'AMA',
       'ANC', 'APN', 'ART', 'ASE', 'ATL', 'ATW', 'ATY', 'AUS', 'AVL',
       'AVP', 'AZA', 'AZO', 'BDL', 'BET', 'BFF', 'BFL', 'BFM', 'BGM',
       'BGR', 'BHM', 'BIL', 'BIS', 'BJI', 'BKG', 'BLI', 'BLV', 'BMI',
       'BNA', 'BOI', 'BOS', 'BPT', 'BQK', 'BQN', 'BRD', 'BRO', 'BRW',
       'BTM', 'BTR', 'BTV', 'BUF', 'BUR', 'BWI', 'BZN', 'CAE', 'CAK',
       'CDC', 'CDV', 'CGI', 'CHA', 'CHO', 'CHS', 'CID', 'CIU', 'CKB',
       'CLE', 'CLL', 'CLT', 'CMH', 'CMI', 'CMX', 'CNY', 'COD', 'COS',
       'COU', 'CPR', 'CRP', 'CRW', 'CSG', 'CVG', 'CWA', 'CYS', 'DAB',
       'DAL', 'DAY', 'DBQ', 'DCA', 'DEN', 'DFW', 'DHN', 'DIK', 'DLG',
       'DLH', 'DRO', 'DRT', 'DSM', 'DTW', 'DUT', 'DVL', 'EAR', 'EAT',
       'EAU', 'ECP', 'EGE', 'EKO', 'ELM', 'ELP', 'ERI', 'ESC', 'EUG',
       'EVV', 'EWN', 'EWR', 'EYW', 'FAI', 'FAR', 'FAT', 'FAY', 'FCA',
       'FLG', 'FLL',

In [29]:
des=flights_train['dest'].unique()
des.sort()
des

array(['ABE', 'ABI', 'ABQ', 'ABR', 'ABY', 'ACK', 'ACT', 'ACV', 'ACY',
       'ADK', 'ADQ', 'AEX', 'AGS', 'AKN', 'ALB', 'ALO', 'ALW', 'AMA',
       'ANC', 'APN', 'ART', 'ASE', 'ATL', 'ATW', 'ATY', 'AUS', 'AVL',
       'AVP', 'AZA', 'AZO', 'BDL', 'BET', 'BFF', 'BFL', 'BFM', 'BGM',
       'BGR', 'BHM', 'BIL', 'BIS', 'BJI', 'BKG', 'BLI', 'BLV', 'BMI',
       'BNA', 'BOI', 'BOS', 'BPT', 'BQK', 'BQN', 'BRD', 'BRO', 'BRW',
       'BTM', 'BTR', 'BTV', 'BUF', 'BUR', 'BWI', 'BZN', 'CAE', 'CAK',
       'CDC', 'CDV', 'CGI', 'CHA', 'CHO', 'CHS', 'CID', 'CIU', 'CKB',
       'CLE', 'CLL', 'CLT', 'CMH', 'CMI', 'CMX', 'CNY', 'COD', 'COS',
       'COU', 'CPR', 'CRP', 'CRW', 'CSG', 'CVG', 'CWA', 'CYS', 'DAB',
       'DAL', 'DAY', 'DBQ', 'DCA', 'DEN', 'DFW', 'DHN', 'DIK', 'DLG',
       'DLH', 'DRO', 'DRT', 'DSM', 'DTW', 'DUT', 'DVL', 'EAR', 'EAT',
       'EAU', 'ECP', 'EGE', 'EKO', 'ELM', 'ELP', 'ERI', 'ESC', 'EUG',
       'EVV', 'EWN', 'EWR', 'EYW', 'FAI', 'FAR', 'FAT', 'FAY', 'FCA',
       'FLG', 'FLL',

In [30]:
len(ori),len(des)

(375, 375)

In [34]:
result = all(x == y for x, y in zip(ori, des))
print(result)


True


In [25]:
flights_train.head(10)

,Unnamed: 0,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,cancellation_code,diverted,dup,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,0,2018-09-20,DL,DL_CODESHARE,DL,7362,OO,N8942A,7362,13459,MQT,"Marquette, MI",13487,MSP,"Minneapolis, MN",731,723.0,-8.0,6.0,729.0,727.0,5.0,749,732.0,-17.0,0,NaN,0,N,78.0,69.0,58.0,1,300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2018-09-20,DL,DL_CODESHARE,DL,7369,OO,N429SW,7369,10333,APN,"Alpena, MI",14150,PLN,"Pellston, MI",528,517.0,-11.0,14.0,531.0,550.0,7.0,613,557.0,-16.0,0,NaN,0,N,45.0,40.0,19.0,1,69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2018-09-20,DL,DL_CODESHARE,DL,7383,OO,N979EV,7383,11122,CPR,"Casper, WY",14869,SLC,"Salt Lake City, UT",1145,1136.0,-9.0,13.0,1149.0,1251.0,5.0,1309,1256.0,-13.0,0,NaN,0,N,84.0,80.0,62.0,1,320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2018-09-20,DL,DL_CODESHARE,DL,7406,OO,N429SW,7406,13459,MQT,"Marquette, MI",11433,DTW,"Detroit, MI",1510,1503.0,-7.0,7.0,1510.0,1607.0,10.0,1635,1617.0,-18.0,0,NaN,0,N,85.0,74.0,57.0,1,349,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2018-09-20,DL,DL_CODESHARE,DL,7408,OO,N465SW,7408,14150,PLN,"Pellston, MI",10333,APN,"Alpena, MI",2150,2247.0,57.0,20.0,2307.0,2337.0,5.0,2229,2342.0,73.0,0,NaN,0,N,39.0,55.0,30.0,1,69,0.0,0.0,16.0,0.0,57.0,NaN,NaN,NaN,NaN
5,5,2018-09-20,DL,DL_CODESHARE,DL,7423,OO,N685BR,7423,10739,BRD,"Brainerd, MN",13487,MSP,"Minneapolis, MN",700,656.0,-4.0,3.0,659.0,724.0,3.0,802,727.0,-35.0,0,NaN,0,N,62.0,31.0,25.0,1,114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,2018-09-20,DL,DL_CODESHARE,DL,7432,OO,N496CA,7432,15389,TWF,"Twin Falls, ID",14869,SLC,"Salt Lake City, UT",1220,1216.0,-4.0,11.0,1227.0,1303.0,5.0,1312,1308.0,-4.0,0,NaN,0,N,52.0,52.0,36.0,1,175,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,2018-09-21,AA,AA,AA,1187,AA,N159AN,1187,11298,DFW,"Dallas/Fort Worth, TX",12953,LGA,"New York, NY",1135,1137.0,2.0,12.0,1149.0,1551.0,4.0,1601,1555.0,-6.0,0,NaN,0,N,206.0,198.0,182.0,1,1389,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,2018-09-21,AA,AA,AA,1189,AA,N922AN,1189,15304,TPA,"Tampa, FL",11298,DFW,"Dallas/Fort Worth, TX",600,559.0,-1.0,15.0,614.0,721.0,13.0,745,734.0,-11.0,0,NaN,0,N,165.0,155.0,127.0,1,929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,2018-09-21,AA,AA,AA,1198,AA,N154AA,1198,14771,SFO,"San Francisco, CA",11298,DFW,"Dallas/Fort Worth, TX",607,600.0,-7.0,35.0,635.0,1201.0,9.0,1143,1210.0,27.0,0,NaN,0,N,216.0,250.0,206.0,1,1464,0.0,0.0,27.0,0.0,0.0,NaN,NaN,NaN,NaN


In [22]:
flights_train['origin_city_name'].value_counts()

Chicago, IL              47972
Atlanta, GA              39556
New York, NY             29701
Dallas/Fort Worth, TX    29475
Denver, CO               27581
                         ...  
Pago Pago, TT                8
Rota, TT                     7
Gustavus, AK                 7
Dillingham, AK               5
Bullhead City, AZ            2
Name: origin_city_name, Length: 367, dtype: int64

In [6]:
flights_train.head()

,Unnamed: 0,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,cancellation_code,diverted,dup,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,0,2018-09-20,DL,DL_CODESHARE,DL,7362,OO,N8942A,7362,13459,MQT,"Marquette, MI",13487,MSP,"Minneapolis, MN",731,723.0,-8.0,6.0,729.0,727.0,5.0,749,732.0,-17.0,0,NaN,0,N,78.0,69.0,58.0,1,300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2018-09-20,DL,DL_CODESHARE,DL,7369,OO,N429SW,7369,10333,APN,"Alpena, MI",14150,PLN,"Pellston, MI",528,517.0,-11.0,14.0,531.0,550.0,7.0,613,557.0,-16.0,0,NaN,0,N,45.0,40.0,19.0,1,69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2018-09-20,DL,DL_CODESHARE,DL,7383,OO,N979EV,7383,11122,CPR,"Casper, WY",14869,SLC,"Salt Lake City, UT",1145,1136.0,-9.0,13.0,1149.0,1251.0,5.0,1309,1256.0,-13.0,0,NaN,0,N,84.0,80.0,62.0,1,320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2018-09-20,DL,DL_CODESHARE,DL,7406,OO,N429SW,7406,13459,MQT,"Marquette, MI",11433,DTW,"Detroit, MI",1510,1503.0,-7.0,7.0,1510.0,1607.0,10.0,1635,1617.0,-18.0,0,NaN,0,N,85.0,74.0,57.0,1,349,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2018-09-20,DL,DL_CODESHARE,DL,7408,OO,N465SW,7408,14150,PLN,"Pellston, MI",10333,APN,"Alpena, MI",2150,2247.0,57.0,20.0,2307.0,2337.0,5.0,2229,2342.0,73.0,0,NaN,0,N,39.0,55.0,30.0,1,69,0.0,0.0,16.0,0.0,57.0,NaN,NaN,NaN,NaN


In [10]:
df_clf=flights_train[['fl_date', 'op_unique_carrier','origin_city_name',  'dest_city_name', 'crs_dep_time','crs_arr_time', 'cancelled']].copy()

In [11]:
df_clf['cancelled'].value_counts()

0    782924
1     13404
Name: cancelled, dtype: int64

In [12]:
missing_values= df_clf.isnull().sum().sort_values(ascending=False)
missing_values

fl_date              0
op_unique_carrier    0
origin_city_name     0
dest_city_name       0
crs_dep_time         0
crs_arr_time         0
cancelled            0
dtype: int64

In [13]:
df_clf=convert_time(df_clf)
df_clf=bins(df_clf)

df_clf.head()

,fl_date,op_unique_carrier,origin_city_name,dest_city_name,crs_dep_time,crs_arr_time,cancelled,month,day_of_year,day_of_week,dep_hour,dep_part_day,season
0,2018-09-20,OO,"Marquette, MI","Minneapolis, MN",0731,749,0,9,263,3,07,1,4
1,2018-09-20,OO,"Alpena, MI","Pellston, MI",0528,613,0,9,263,3,05,0,4
2,2018-09-20,OO,"Casper, WY","Salt Lake City, UT",1145,1309,0,9,263,3,11,2,4
3,2018-09-20,OO,"Marquette, MI","Detroit, MI",1510,1635,0,9,263,3,15,2,4
4,2018-09-20,OO,"Pellston, MI","Alpena, MI",2150,2229,0,9,263,3,21,4,4


In [14]:
df_clf.columns

Index(['fl_date', 'op_unique_carrier', 'origin_city_name', 'dest_city_name',
       'crs_dep_time', 'crs_arr_time', 'cancelled', 'month', 'day_of_year',
       'day_of_week', 'dep_hour', 'dep_part_day', 'season'],
      dtype='object')

In [15]:
X=df_clf[['op_unique_carrier', 'origin_city_name', 'dest_city_name',
       'month', 'day_of_year', 'day_of_week',
       'dep_hour', 'dep_part_day', 'season']]
y = df_clf.cancelled


In [16]:
from sklearn.model_selection import train_test_split


In [17]:
cat_dummies = pd.get_dummies( X[["op_unique_carrier", "origin_city_name", "dest_city_name"]])
X = pd.concat([X,cat_dummies] , axis=1)



In [18]:
X.drop(['op_unique_carrier', 'origin_city_name', 'dest_city_name'], axis=1, inplace=True)
X


,month,day_of_year,day_of_week,dep_hour,dep_part_day,season,op_unique_carrier_9E,op_unique_carrier_9K,op_unique_carrier_AA,op_unique_carrier_AS,op_unique_carrier_AX,op_unique_carrier_B6,op_unique_carrier_C5,op_unique_carrier_CP,op_unique_carrier_DL,op_unique_carrier_EM,op_unique_carrier_EV,op_unique_carrier_F9,op_unique_carrier_G4,op_unique_carrier_G7,op_unique_carrier_HA,op_unique_carrier_KS,op_unique_carrier_MQ,op_unique_carrier_NK,op_unique_carrier_OH,op_unique_carrier_OO,op_unique_carrier_PT,op_unique_carrier_QX,op_unique_carrier_UA,op_unique_carrier_VX,op_unique_carrier_WN,op_unique_carrier_YV,op_unique_carrier_YX,op_unique_carrier_ZW,"origin_city_name_Aberdeen, SD","origin_city_name_Abilene, TX","origin_city_name_Adak Island, AK","origin_city_name_Aguadilla, PR","origin_city_name_Akron, OH","origin_city_name_Albany, GA","origin_city_name_Albany, NY","origin_city_name_Albuquerque, NM","origin_city_name_Alexandria, LA","origin_city_name_Allentown/Bethlehem/Easton, PA","origin_city_name_Alpena, MI","origin_city_name_Amarillo, TX","origin_city_name_Anchorage, AK","origin_city_name_Appleton, WI","origin_city_name_Arcata/Eureka, CA","origin_city_name_Asheville, NC","origin_city_name_Ashland, WV","origin_city_name_Aspen, CO","origin_city_name_Atlanta, GA","origin_city_name_Atlantic City, NJ","origin_city_name_Augusta, GA","origin_city_name_Austin, TX","origin_city_name_Bakersfield, CA","origin_city_name_Baltimore, MD","origin_city_name_Bangor, ME","origin_city_name_Barrow, AK","origin_city_name_Baton Rouge, LA","origin_city_name_Beaumont/Port Arthur, TX","origin_city_name_Belleville, IL","origin_city_name_Bellingham, WA","origin_city_name_Bemidji, MN","origin_city_name_Bend/Redmond, OR","origin_city_name_Bethel, AK","origin_city_name_Billings, MT","origin_city_name_Binghamton, NY","origin_city_name_Birmingham, AL","origin_city_name_Bismarck/Mandan, ND","origin_city_name_Bloomington/Normal, IL","origin_city_name_Boise, ID","origin_city_name_Boston, MA","origin_city_name_Bozeman, MT","origin_city_name_Brainerd, MN","origin_city_name_Branson, MO","origin_city_name_Bristol/Johnson City/Kingsport, TN","origin_city_name_Brownsville, TX","origin_city_name_Brunswick, GA","origin_city_name_Buffalo, NY","origin_city_name_Bullhead City, AZ","origin_city_name_Burbank, CA","origin_city_name_Burlington, VT","origin_city_name_Butte, MT","origin_city_name_Cape Girardeau, MO","origin_city_name_Casper, WY","origin_city_name_Cedar City, UT","origin_city_name_Cedar Rapids/Iowa City, IA","origin_city_name_Champaign/Urbana, IL","origin_city_name_Charleston, SC","origin_city_name_Charleston/Dunbar, WV","origin_city_name_Charlotte Amalie, VI","origin_city_name_Charlotte, NC","origin_city_name_Charlottesville, VA","origin_city_name_Chattanooga, TN","origin_city_name_Cheyenne, WY","origin_city_name_Chicago, IL","origin_city_name_Christiansted, VI","origin_city_name_Cincinnati, OH","origin_city_name_Clarksburg/Fairmont, WV","origin_city_name_Cleveland, OH","origin_city_name_Cody, WY","origin_city_name_College Station/Bryan, TX","origin_city_name_Colorado Springs, CO","origin_city_name_Columbia, MO","origin_city_name_Columbia, SC","origin_city_name_Columbus, GA","origin_city_name_Columbus, MS","origin_city_name_Columbus, OH","origin_city_name_Concord, NC","origin_city_name_Cordova, AK","origin_city_name_Corpus Christi, TX","origin_city_name_Dallas, TX","origin_city_name_Dallas/Fort Worth, TX","origin_city_name_Dayton, OH","origin_city_name_Daytona Beach, FL","origin_city_name_Deadhorse, AK","origin_city_name_Del Rio, TX","origin_city_name_Denver, CO","origin_city_name_Des Moines, IA","origin_city_name_Detroit, MI","origin_city_name_Devils Lake, ND","origin_city_name_Dickinson, ND","origin_city_name_Dillingham, AK","origin_city_name_Dothan, AL","origin_city_name_Dubuque, IA","origin_city_name_Duluth, MN","origin_city_name_Durango, CO","origin_city_name_Eagle, CO","origin_city_name_Eau Claire, WI","origin_city_name_El Paso, TX","origin_city_name_Elko, NV","

In [19]:
X.columns

Index(['month', 'day_of_year', 'day_of_week', 'dep_hour', 'dep_part_day',
       'season', 'op_unique_carrier_9E', 'op_unique_carrier_9K',
       'op_unique_carrier_AA', 'op_unique_carrier_AS',
       ...
       'dest_city_name_Wichita Falls, TX', 'dest_city_name_Wichita, KS',
       'dest_city_name_Williamsport, PA', 'dest_city_name_Williston, ND',
       'dest_city_name_Wilmington, NC', 'dest_city_name_Worcester, MA',
       'dest_city_name_Wrangell, AK', 'dest_city_name_Yakima, WA',
       'dest_city_name_Yakutat, AK', 'dest_city_name_Yuma, AZ'],
      dtype='object', length=768)

Due to huge imbalance in the data, smote technique is used

In [23]:

from sklearn.linear_model import LogisticRegression
lr_clf=LogisticRegression()
from sklearn.ensemble import RandomForestClassifier
rf_clf=RandomForestClassifier()
from xgboost import XGBClassifier
xgb_clf=XGBClassifier()
from sklearn.metrics import roc_curve, auc, confusion_matrix,accuracy_score,classification_report
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler


In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=10)

In [52]:
#oversample = SMOTE()
#X_smote,y_smote=oversample.fit_resample(X_train,y_train)

In [35]:
undersample = RandomUnderSampler()
X_under,y_under=undersample.fit_resample(X_train,y_train)

In [36]:
y_train.value_counts()

0    548091
1      9338
Name: cancelled, dtype: int64

In [53]:
#y_smote.value_counts()

In [37]:
y_under.value_counts()

0    9338
1    9338
Name: cancelled, dtype: int64

In [38]:
lr_clf.fit(X_train,y_train)

/Users/phanisingaraju/opt/anaconda3/envs/flights_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [39]:
ypred_lr=lr_clf.predict(X_test)

In [40]:
print("Accuracy on training set LR: {:.3f}".format(lr_clf.score(X_train,y_train)))
print("Accuracy on testing set LR: {:.3f}".format(accuracy_score(y_test,ypred_lr)))
print("Classification report of LR:\n", classification_report(y_test,ypred_lr))

Accuracy on training set LR: 0.983
Accuracy on testing set LR: 0.983
Classification report of LR:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99    234833
           1       0.00      0.00      0.00      4066

    accuracy                           0.98    238899
   macro avg       0.49      0.50      0.50    238899
weighted avg       0.97      0.98      0.97    238899



/Users/phanisingaraju/opt/anaconda3/envs/flights_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/phanisingaraju/opt/anaconda3/envs/flights_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/phanisingaraju/opt/anaconda3/envs/flights_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beh

In [192]:
#pickle.dump(lr_clf, open('../models/lr_clf.pkl', 'wb'))


In [193]:
#lr_clf_smote=LogisticRegression()


In [194]:
#lr_clf_smote.fit(X_smote,y_smote)

/Users/phanisingaraju/opt/anaconda3/envs/flights_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [195]:
#ypred_lr_smote=lr_clf_smote.predict(X_test)

In [54]:
#print("Accuracy on training set LR_smote: {:.3f}".format(lr_clf_smote.score(X_smote,y_smote)))
#print("Accuracy on testing set LR_smote: {:.3f}".format(accuracy_score(y_test,ypred_lr_smote)))
#print("Classification report of LR_smote:\n", classification_report(y_test,ypred_lr_smote))

In [55]:
#pickle.dump(lr_clf_smote, open('../models/lr_clf_smote.pkl', 'wb'))


In [56]:
#cm=confusion_matrix(y_test, ypred_lr)
#sns.heatmap(cm, annot=True,
#                    cmap=plt.cm.Blues,
#                   xticklabels=['No cancellation', 'cancellation'],
 #                  yticklabels=['No cancellation', 'cancellation']).set_title('Logistic Regression ')

In [57]:
#cm=confusion_matrix(y_test, ypred_lr_smote)
#sns.heatmap(cm, annot=True,
##                    cmap=plt.cm.Blues,
#                   xticklabels=['No cancellation', 'cancellation'],
 #                  yticklabels=['No cancellation', 'cancellation']).set_title('Logistic Regression with smote oversampling')

In [41]:
lr_clf_under=LogisticRegression()
lr_clf_under.fit(X_under,y_under)

/Users/phanisingaraju/opt/anaconda3/envs/flights_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [42]:
ypred_lr_under=lr_clf_under.predict(X_test)

In [43]:
print("Accuracy on training set LR_under: {:.3f}".format(lr_clf_under.score(X_under,y_under)))
print("Accuracy on testing set LR_under: {:.3f}".format(accuracy_score(y_test,ypred_lr_under)))
print("Classification report of LR_under:\n", classification_report(y_test,ypred_lr_under))

Accuracy on training set LR_under: 0.663
Accuracy on testing set LR_under: 0.613
Classification report of LR_under:
               precision    recall  f1-score   support

           0       0.99      0.61      0.76    234833
           1       0.03      0.71      0.06      4066

    accuracy                           0.61    238899
   macro avg       0.51      0.66      0.41    238899
weighted avg       0.98      0.61      0.74    238899



In [58]:
#resampling manually

In [44]:
df_can = df_clf[df_clf.cancelled == 1 ]
df_nocan = df_clf[df_clf.cancelled == 0 ]
df_nocan = df_nocan.sample(n=13404, random_state=10)
df_half = pd.concat([df_can,df_nocan])
df_half = shuffle(df_half)

In [45]:
X=df_half[['op_unique_carrier', 'origin', 'dest',
       'month', 'day_of_year', 'day_of_week',
       'dep_hour', 'dep_part_day', 'season']]
y = df_half.cancelled

In [46]:
cat_dummies = pd.get_dummies( X[["op_unique_carrier", "origin", "dest"]])
X = pd.concat([X,cat_dummies] , axis=1)
X.drop(['op_unique_carrier', 'origin', 'dest'], axis=1, inplace=True)


In [61]:
X['dep_hour'] = pd.to_numeric(X['dep_hour'])


In [62]:
X.dtypes

month           int64
day_of_year     int64
day_of_week     int64
dep_hour        int64
dep_part_day    int64
                ...  
dest_WRG        uint8
dest_XNA        uint8
dest_YAK        uint8
dest_YKM        uint8
dest_YUM        uint8
Length: 744, dtype: object

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=12)

In [64]:
lr_clf=LogisticRegression()
rf_clf=RandomForestClassifier(n_estimators=50, max_depth=10, random_state=25)
xgb_clf=XGBClassifier()


In [65]:
lr_clf.fit(X_train,y_train)
rf_clf.fit(X_train,y_train)
xgb_clf.fit(X_train,y_train)

/Users/phanisingaraju/opt/anaconda3/envs/flights_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [66]:
ypred_lr=lr_clf.predict(X_test)
ypred_rf=rf_clf.predict(X_test)
ypred_xgb=xgb_clf.predict(X_test)

In [67]:
print("Accuracy on training set LR: {:.3f}".format(lr_clf.score(X_train,y_train)))
print("Accuracy on testing set LR: {:.3f}".format(accuracy_score(y_test,ypred_lr)))
print("Classification report of LR:\n", classification_report(y_test,ypred_lr))
print("Accuracy on training set RF: {:.3f}".format(rf_clf.score(X_train,y_train)))
print("Accuracy on testing set RF: {:.3f}".format(accuracy_score(y_test,ypred_rf)))
print("Classification report of RF:\n", classification_report(y_test,ypred_rf))
print("Accuracy on training set XGB: {:.3f}".format(xgb_clf.score(X_train,y_train)))
print("Accuracy on testing set XGB: {:.3f}".format(accuracy_score(y_test,ypred_xgb)))
print("Classification report of XGB:\n", classification_report(y_test,ypred_xgb))

Accuracy on training set LR: 0.666
Accuracy on testing set LR: 0.661
Classification report of LR:
               precision    recall  f1-score   support

           0       0.67      0.63      0.65      4005
           1       0.65      0.69      0.67      4038

    accuracy                           0.66      8043
   macro avg       0.66      0.66      0.66      8043
weighted avg       0.66      0.66      0.66      8043

Accuracy on training set RF: 0.675
Accuracy on testing set RF: 0.648
Classification report of RF:
               precision    recall  f1-score   support

           0       0.68      0.55      0.61      4005
           1       0.62      0.75      0.68      4038

    accuracy                           0.65      8043
   macro avg       0.65      0.65      0.64      8043
weighted avg       0.65      0.65      0.64      8043

Accuracy on training set XGB: 0.800
Accuracy on testing set XGB: 0.716
Classification report of XGB:
               precision    recall  f1-score   

In [68]:
pickle.dump(lr_clf, open('../models/lr_app_clf_sampling.pkl', 'wb'))
pickle.dump(rf_clf, open('../models/rf_app_clf_sampling.pkl', 'wb'))
pickle.dump(xgb_clf, open('../models/xgb_app_clf_sampling.pkl', 'wb'))


### To predict the delay in the flight

In [69]:
flights_train.columns

Index(['Unnamed: 0', 'fl_date', 'mkt_unique_carrier', 'branded_code_share',
       'mkt_carrier', 'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'dep_time',
       'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in',
       'crs_arr_time', 'arr_time', 'arr_delay', 'cancelled',
       'cancellation_code', 'diverted', 'dup', 'crs_elapsed_time',
       'actual_elapsed_time', 'air_time', 'flights', 'distance',
       'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay', 'first_dep_time', 'total_add_gtime',
       'longest_add_gtime', 'no_name'],
      dtype='object')

In [74]:
df_reg=flights_train.loc[flights_train['cancelled']==0]

In [75]:
df_reg=df_reg[['fl_date','op_unique_carrier','origin','dest','crs_dep_time','dep_delay']]

In [77]:
df_reg=df_reg.dropna()

In [78]:
missing_values= df_reg.isnull().sum().sort_values(ascending=False)
missing_values

fl_date              0
op_unique_carrier    0
origin               0
dest                 0
crs_dep_time         0
dep_delay            0
dtype: int64

In [79]:
df_reg=convert_time(df_reg)
df_reg=bins(df_reg)

df_reg.head()

,fl_date,op_unique_carrier,origin,dest,crs_dep_time,dep_delay,month,day_of_year,day_of_week,dep_hour,dep_part_day,season
0,2018-09-20,OO,MQT,MSP,0731,-8.0,9,263,3,07,1,4
1,2018-09-20,OO,APN,PLN,0528,-11.0,9,263,3,05,0,4
2,2018-09-20,OO,CPR,SLC,1145,-9.0,9,263,3,11,2,4
3,2018-09-20,OO,MQT,DTW,1510,-7.0,9,263,3,15,2,4
4,2018-09-20,OO,PLN,APN,2150,57.0,9,263,3,21,4,4


In [81]:
df_reg.columns

Index(['fl_date', 'op_unique_carrier', 'origin', 'dest', 'crs_dep_time',
       'dep_delay', 'month', 'day_of_year', 'day_of_week', 'dep_hour',
       'dep_part_day', 'season'],
      dtype='object')

In [82]:
X=df_reg[['op_unique_carrier', 'origin', 'dest',
       'month', 'day_of_year', 'day_of_week',
       'dep_hour', 'dep_part_day', 'season']]
y = df_reg.dep_delay

In [83]:
cat_dummies = pd.get_dummies( X[["op_unique_carrier", "origin", "dest"]])
X = pd.concat([X,cat_dummies] , axis=1)
X.drop(['op_unique_carrier', 'origin', 'dest'], axis=1, inplace=True)
X


,month,day_of_year,day_of_week,dep_hour,dep_part_day,season,op_unique_carrier_9E,op_unique_carrier_9K,op_unique_carrier_AA,op_unique_carrier_AS,op_unique_carrier_AX,op_unique_carrier_B6,op_unique_carrier_C5,op_unique_carrier_CP,op_unique_carrier_DL,op_unique_carrier_EM,op_unique_carrier_EV,op_unique_carrier_F9,op_unique_carrier_G4,op_unique_carrier_G7,op_unique_carrier_HA,op_unique_carrier_KS,op_unique_carrier_MQ,op_unique_carrier_NK,op_unique_carrier_OH,op_unique_carrier_OO,op_unique_carrier_PT,op_unique_carrier_QX,op_unique_carrier_UA,op_unique_carrier_VX,op_unique_carrier_WN,op_unique_carrier_YV,op_unique_carrier_YX,op_unique_carrier_ZW,origin_ABE,origin_ABI,origin_ABQ,origin_ABR,origin_ABY,origin_ACK,origin_ACT,origin_ACV,origin_ACY,origin_ADK,origin_ADQ,origin_AEX,origin_AGS,origin_AKN,origin_ALB,origin_ALO,origin_ALW,origin_AMA,origin_ANC,origin_APN,origin_ART,origin_ASE,origin_ATL,origin_ATW,origin_ATY,origin_AUS,origin_AVL,origin_AVP,origin_AZA,origin_AZO,origin_BDL,origin_BET,origin_BFF,origin_BFL,origin_BFM,origin_BGM,origin_BGR,origin_BHM,origin_BIL,origin_BIS,origin_BJI,origin_BKG,origin_BLI,origin_BLV,origin_BMI,origin_BNA,origin_BOI,origin_BOS,origin_BPT,origin_BQK,origin_BQN,origin_BRD,origin_BRO,origin_BRW,origin_BTM,origin_BTR,origin_BTV,origin_BUF,origin_BUR,origin_BWI,origin_BZN,origin_CAE,origin_CAK,origin_CDC,origin_CDV,origin_CGI,origin_CHA,origin_CHO,origin_CHS,origin_CID,origin_CIU,origin_CKB,origin_CLE,origin_CLL,origin_CLT,origin_CMH,origin_CMI,origin_CMX,origin_CNY,origin_COD,origin_COS,origin_COU,origin_CPR,origin_CRP,origin_CRW,origin_CSG,origin_CVG,origin_CWA,origin_CYS,origin_DAB,origin_DAL,origin_DAY,origin_DBQ,origin_DCA,origin_DEN,origin_DFW,origin_DHN,origin_DIK,origin_DLG,origin_DLH,origin_DRO,origin_DRT,origin_DSM,origin_DTW,origin_DUT,origin_DVL,origin_EAR,origin_EAT,origin_EAU,origin_ECP,origin_EGE,origin_EKO,origin_ELM,origin_ELP,origin_ERI,origin_ESC,origin_EUG,origin_EVV,origin_EWN,origin_EWR,origin_EYW,origin_FAI,origin_FAR,origin_FAT,origin_FAY,origin_FCA,origin_FLG,origin_FLL,origin_FLO,origin_FNT,origin_FSD,origin_FSM,origin_FWA,origin_GCC,origin_GCK,origin_GEG,origin_GFK,origin_GGG,origin_GJT,origin_GNV,origin_GPT,origin_GRB,origin_GRI,origin_GRK,origin_GRR,origin_GSO,origin_GSP,origin_GST,origin_GTF,origin_GTR,origin_GUC,origin_GUM,origin_HDN,origin_HGR,origin_HHH,origin_HIB,origin_HLN,origin_HNL,origin_HOB,origin_HOU,origin_HPN,origin_HRL,origin_HSV,origin_HTS,origin_HVN,origin_HYA,origin_HYS,origin_IAD,origin_IAG,origin_IAH,origin_ICT,origin_IDA,origin_IFP,origin_ILM,origin_IMT,origin_IND,origin_INL,origin_IPT,origin_ISN,origin_ISP,origin_ITH,origin_ITO,origin_JAC,origin_JAN,origin_JAX,origin_JFK,origin_JHM,origin_JLN,origin_JMS,origin_JNU,origin_KOA,origin_KTN,origin_LAN,origin_LAR,origin_LAS,origin_LAW,origin_LAX,origin_LBB,origin_LBE,origin_LBF,origin_LBL,origin_LCH,origin_LCK,origin_LEX,origin_LFT,origin_LGA,origin_LGB,origin_LIH,origin_LIT,origin_LNK,origin_LNY,origin_LRD,origin_LSE,origin_LWB,origin_LWS,origin_LYH,origin_MAF,origin_MBS,origin_MCI,origin_MCO,origin_MDT,origin_MDW,origin_MEI,origin_MEM,origin_MFE,origin_MFR,origin_MGM,origin_MHK,origin_MHT,origin_MIA,origin_MKE,origin_MKG,origin_MKK,origin_MLB,origin_MLI,origin_MLU,origin_MMH,origin_MOB,origin_MOT,origin_MQT,origin_MRY,origin_MSN,origin_MSO,origin_MSP,origin_MSY,origin_MTJ,origin_MVY,origin_MYR,origin_OAJ,origin_OAK,origin_OGD,origin_OGG,origin_OGS,origin_OKC,origin_OMA,origin_OME,origin_ONT,origin_ORD,origin_ORF,origin_ORH,origin_OTH,origin_OTZ,origin_OWB,origin_PAE,origin_PAH,origin_PBG,origin_PBI,origin_PDX,origin_PGD,origin_PGV,origin_PHF,origin_PHL,origin_PHX,origin_PIA,origin_PIB,origin_PIE,origin_PIH,origin_PIR,origin_PIT,origin_PLN,origin_PNS,origin_PPG,origin_PQI,origin_PRC,origin_PSC,origin_PSE,origin_PSG,origin_PSM,origin_PSP,origin_PUB,origin_PUW,origin_PVD,origin_PVU,origin_PWM,origin_RAP,origin_RDD,origin_RDM,origin_RDU,origin_RFD,origin_RHI,origin_RIC,origin_RKS,origin_RNO,origin_ROA,origin_R

In [90]:
X.columns

Index(['month', 'day_of_year', 'day_of_week', 'dep_hour', 'dep_part_day',
       'season', 'op_unique_carrier_9E', 'op_unique_carrier_9K',
       'op_unique_carrier_AA', 'op_unique_carrier_AS',
       ...
       'dest_VEL', 'dest_VLD', 'dest_VPS', 'dest_WRG', 'dest_WYS', 'dest_XNA',
       'dest_XWA', 'dest_YAK', 'dest_YKM', 'dest_YUM'],
      dtype='object', length=784)

9E- Endeavor Air Inc,9K-Cape Air
,AA-American Airlines Inc,AS-Alaska Airlines Inc,AX-Trans States Airlines,B6-JetBlue Airways,C5-Commutair Aka Champlain Enterprises,CP-Compass Airlines,
DL-	Delta Air Lines Inc,EM-Empire Airlines Inc,EV-ExpressJet Airlines Inc,F9-	Frontier Airlines Inc,G4-Allegiant Air,
G7-GoJet Airlines LLC d/b/a United Express,HA-	Hawaiian Airlines Inc,KS-	Peninsula Airways Inc,
MQ-	Envoy Air,NK-Spirit Air Lines,OH-	PSA Airlines Inc,OO-SkyWest Airlines Inc,PT-Piedmont Airlines,
QX-Horizon Air,UA-United Air Lines Inc,VX-Virgin America,WN-Southwest Airlines Co,YV-	Mesa Airlines Inc,YX-Republic Airways,
ZW-	Air Wisconsin Airlines Corp

In [84]:
X['dep_hour'] = pd.to_numeric(X['dep_hour'])


In [85]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score,mean_absolute_error,mean_squared_error,r2_score


In [86]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=9)

In [87]:
rf = RandomForestRegressor(n_estimators=60, max_depth=6, random_state=10)
rf.fit(X_train,y_train)

RandomForestRegressor(max_depth=6, n_estimators=60, random_state=10)

In [88]:
r_sq_rf = rf.score(X_train,y_train)
print('coefficient of determination:', r_sq_rf)

coefficient of determination: 0.031150941866013415


In [89]:
y_pred_rf=rf.predict(X_test)
r2_score(y_test,y_pred_rf)

0.018287225662073858